In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [ ]:
import os
os.chdir('/content/BA')

In [ ]:
!mkdir /content/dataset   #kopieren der featurevektoren um schneller trainiren zu können
!cp -R /content/gdrive/MyDrive/Datasets/channelisland/features /content/dataset/features

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class Test_Dataset(Dataset):
  def __init__(self, annotations_file, dataset_dir):  
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file) 
    self.dataset = dataset_dir
    self.ann_ids = self.coco.getAnnIds()
    self.img_ids = self.coco.getImgIds()

  def __len__(self):
    return len(self.ann_ids)

  def __getitem__(self, idx):
    ann_id = self.ann_ids[idx]
    ann = self.coco.loadAnns([ann_id])
    img_id = ann[0]['image_id']
    img = self.coco.loadImgs([img_id])
    fileN = img[0]['file_name'].split('/')
    classN = fileN[0]
    name = classN.split('.')[0]
    feature_path = self.dataset + '/features/' + name + '_feature.pt'
    features = torch.load(feature_path, map_location=torch.device('cpu'))
    ground_truth = ann[0]['category_id']
    gt_vect = np.zeros(4)
    gt_vect[ground_truth-1] = 1
    gt_vect = torch.tensor(gt_vect)
    return features, ground_truth, ann[0]

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class Train_Dataset(Dataset):
  def __init__(self, annotations_file, dataset_dir):  #taxonomy must be: kingdom phylum class order family genus name
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file)  
    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()

  def __len__(self):
    return len(self.img_ids)

  def __getitem__(self, idx):
    img_id = self.img_ids[idx]
    img = self.coco.loadImgs([img_id])
    ann_ids = self.coco.getAnnIds([img_id])
    anns = self.coco.loadAnns(ann_ids)
    fileN = img[0]['file_name'].split('/')
    classN = fileN[0]
    direcN = fileN[1]
    imgN = fileN[2].split('.')[0]
    feature_path = self.dataset + '/features/' + classN + '/' + direcN + '/' + imgN + '_feature.pt'
    features = torch.load(feature_path, map_location=torch.device('cpu'))
    ground_truth = 0
    if len(anns) > 0:
      ground_truth = anns[0]['category_id']
    gt_vect = np.zeros(4)
    gt_vect[ground_truth-1] = 1
    gt_vect = torch.tensor(gt_vect)

    return features, gt_vect

In [ ]:
dataset_path = '/content/dataset'

test_path = 'PATH_TO_DATASET'

trainset = Train_Dataset('TRAIN_ANNOTATIONS', dataset_path)
testset = Test_Dataset('TEST_ANNOTATIONS', test_path)

In [ ]:
import random
random.seed(7)

train_ids = np.arange(0,trainset.__len__(),1)
test_ids = np.arange(0,testset.__len__(),1)

modelle


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP_net(nn.Module):

    def __init__(self, input_size, output_size):
      super(MLP_net, self).__init__()

      self.lin1 = nn.Linear(input_size, input_size, bias=True)
      self.lin2 = nn.Linear(input_size, input_size, bias=True)
      #self.lin2plus = nn.Linear(input_size, input_size, bias=True)
      #self.lin2plusplus = nn.Linear(input_size, input_size, bias=True)
      #self.lin3 = nn.Linear(input_size, int(input_size/2), bias=True)
      self.lin3plus = nn.Linear(int(input_size/2), int(input_size/2), bias=True)
      self.lin3plus1 = nn.Linear(int(input_size/2), int(input_size/2), bias=True)
      self.lin4 = nn.Linear(int(input_size/2), output_size, bias=True)

    def forward(self, x):
      x = self.lin1(x)
      x = F.relu(x)
      x = self.lin2(x)
      x = F.relu(x)
      #x = self.lin2plus(x)
      #x = F.relu(x)
      #x = self.lin2plusplus(x)
      #x = F.relu(x)
      #x = self.lin3(x)
      #x = F.relu(x)
      x = self.lin3plus(x)
      x = F.relu(x)
      x = self.lin3plus1(x)
      x = F.relu(x)
      x = self.lin4(x)
      return x

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

dataset_path = '/content/dataset'

trainset = Train_Dataset('TRAINSET_ANNOTATIONS.json', dataset_path)

batch_size = 64

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
net = MLP_net(768, 4) #anzahl der klassen ist 4

net.to(device)

#PATH = 'PATH_TO_MODEL'
#net.load_state_dict(torch.load(PATH))
net.train()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [ ]:
loss_graph = []

for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, mask = data
        inputs = inputs.to(device)
        numb_inst = inputs.shape[0]
        mask = mask.to(device)
        mask = mask.type(torch.float32)
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        outputs = outputs.reshape((numb_inst, 4))
        loss = criterion(outputs, mask)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        output_value = 5000

        if i % output_value == output_value - 1:    # print every out_value mini-batches
          print('[%d, %5d] loss: %.20f' %
              (epoch + 1, i + 1, running_loss / output_value))
          
          loss_graph.append(running_loss / output_value)

          running_loss = 0.0
          PATH = '/content/gdrive/MyDrive/models/channelislands/MLP' + str(i) + 'model_mlp.pth'
          #torch.save(net.state_dict(), PATH) #entfernt weil zu langsam

print('Finished Training')

PATH = 'MODEL_SAVE.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
import matplotlib.pyplot as plt

loss_arr = np.asarray(loss_graph)
x_val = np.arange(0, len(loss_graph), 1)

plt.plot(x_val, loss_arr)
plt.ylabel('loss')
plt.show()

**Testing**


In [ ]:
#net = MLP_net(768, 4)

#PATH = 'MODEL_PATH.pth'
#net.to(device)
#net.load_state_dict(torch.load(PATH))
net.eval()

In [ ]:
drive.flush_and_unmount()

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
Ygt = []
Ypred = []

pred_anns = []

net.eval()

for i in test_ids:
  print(i)
  features, gt, annotat = testset.__getitem__(i)

  #features = features.reshape(1, -1)
  features = features.reshape(768)

  #print(features[:4])

  #npfeatures = np.asarray(features)

  Ygt.append(np.asarray(gt, dtype=np.float32))
  features = features.to(device)
  pred = net(features)

  pred = F.softmax(pred, dim = 0)

  pred = pred.detach()
  pred = pred.to('cpu')

  prediction = np.asarray(pred, dtype=np.float32)

  top = np.amax(prediction)
  index = np.where(prediction == top)
  prediction = int(index[0] + 1)

  annotat['mlp_pred'] =  prediction
  pred_anns.append(annotat)

  Ypred.append(np.asarray(pred, dtype=np.float32))

In [ ]:
preds = [p['mlp_pred'] for p in pred_anns]
gt = [p['category_id'] for p in pred_anns]

In [ ]:
import json 

output_file = 'RESULT_FILE.json'

with open(output_file, 'w') as file:
#  json.dump(pred_anns, file, indent = 4, sort_keys=False)

**weitere verarbeitung für metriken**

In [ ]:
YclassesGt = Ygt 
YclassesPred = []

for elem in Ypred:
  top = np.amax(elem)
  index = np.where(elem == top)
  YclassesPred.append(int(index[0])+1)

In [ ]:
YclassesGt = [int(Y) for Y in YclassesGt]

In [ ]:
Ypred = YclassesPred
Ygt = YclassesGt

In [ ]:
from sklearn.metrics import confusion_matrix as confusion

matr = confusion(Ygt, Ypred)  #confusionmatrix

In [ ]:
norm_matrix = np.zeros(matr.shape)
for i in range(matr.shape[0]):
  for j in range(matr.shape[1]):
    norm_matrix[i,j] = matr[i,j]/np.sum(matr[i,:])

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

#confusionmatrix in schön
disp = ConfusionMatrixDisplay(matr)
disp.plot()
disp1 = ConfusionMatrixDisplay(norm_matrix)
disp1.plot()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score

#metriken (accuracy, precision, recall)
accuracy = accuracy_score(Ygt, Ypred)
#avg_prec = average_precision_score(Ygt, Ypred)
f1 = f1_score(Ygt, Ypred, average = None)
precision = precision_score(Ygt, Ypred, average = None)
recall = recall_score(Ygt, Ypred, average = None)

In [ ]:
print(accuracy)
print(f1)
print(precision)
print(recall)